# Transfer Optimisation Notebook

In [ ]:
import pandas as pd
import ipywidgets as widgets
from data_load import load_bank_master, load_fee_table, load_balance, load_cashflow
from safety import calc_safety
from optimise import build_model


In [ ]:
horizon = widgets.IntSlider(value=30, min=1, max=90, description='horizon')
quant = widgets.FloatSlider(value=0.95, min=0.5, max=0.99, step=0.01, description='quantile')
lam = widgets.FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description='lambda')
display(horizon, quant, lam)

In [ ]:
df_master = load_bank_master('bank_master.csv')
df_fee = load_fee_table('fee_table.csv')
df_balance = load_balance('balance_snapshot.csv')
df_flow = load_cashflow('cashflow_history.csv')
safety = calc_safety(df_flow, horizon_days=horizon.value, quantile=quant.value)
model, vars = build_model(df_master, df_fee, df_balance, safety, lam=lam.value)
display(model)